In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

In [2]:
from brio.utils.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from pickle import dump, load
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from functools import reduce


In [6]:
from brio.bias.FreqVsFreqBiasDetector import FreqVsFreqBiasDetector, BiasDetector
from brio.risk.HazardFromBiasDetectionCalculator import HazardFromBiasDetectionCalculator
from brio.risk.RiskCalculator import RiskCalculator

In [ ]:
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric



In [ ]:
input_data_path = "../data/CRIF_POC/"
output_data_path = "../data/output/"
total = pd.read_csv(input_data_path + 'german_credit_crif.csv')

In [ ]:
def process_german_credit(df_german):
    df_num = df_german.copy()
    gender_dic={'A91': 1, 'A92': 2, 'A93': 1, 'A94': 1, 'A95': 2} #1=male, 2=female
    personal_status_dic={'A91': 2, 'A92': 2, 'A93': 1, 'A94': 2, 'A95': 1} #1=single, 2=married/separated/widowed/divorced

    df_num['age_group']=pd.cut(df_german["age"], #todo change in case
       bins=[0, 27, 37, 47, 100],
       labels=[1, 2, 3, 4]).astype('int64')
    df_num['gender']=df_german['personal_status_and_sex'].map(gender_dic)
    df_num['marital_status']=df_german['personal_status_and_sex'].map(personal_status_dic)

    attribute1_dic={'A11': 1, 'A12': 2, 'A13': 3, 'A14': 4}
    attribute3_dic={'A30': 0, 'A31': 1, 'A32': 2, 'A33': 3, 'A34': 4}
    attribute4_dic={'A40': 0, 'A41': 1, 'A42': 2, 'A43': 3, 'A44': 4, 'A45': 5, 'A46': 6, 'A47': 7, 'A48': 8, 'A49': 9, 'A410': 10}
    attribute6_dic={'A61': 1, 'A62': 2, 'A63': 3, 'A64': 4, 'A65': 5}
    attribute7_dic={'A71': 1, 'A72': 2, 'A73': 3, 'A74': 4, 'A75': 5}
    attribute9_dic={'A91': 1, 'A92': 2, 'A93': 3, 'A94': 4, 'A95': 5}
    attribute10_dic={'A101': 1, 'A102': 2, 'A103': 3}
    attribute12_dic={'A121': 1, 'A122': 2, 'A123': 3, 'A124': 4}
    attribute14_dic={'A141': 1, 'A142': 2, 'A143': 3}
    attribute15_dic={'A151': 1, 'A152': 2, 'A153': 3}
    attribute17_dic={'A171': 1, 'A172': 2, 'A173': 3, 'A174': 4}
    attribute19_dic={'A191': 1, 'A192': 2}
    attribute20_dic={'A201': 1, 'A202': 2}


    
    df_num['status']=df_german['status'].map(attribute1_dic)
    df_num['duration']=df_german['duration']
    df_num['credit_history']=df_german['credit_history'].map(attribute3_dic)
    df_num['purpose']=df_german['purpose'].map(attribute4_dic)
    df_num['credit_amount']=df_german['credit_amount']
    df_num['savings/bonds']=df_german['savings/bonds'].map(attribute6_dic)
    df_num['employment_duration']=df_german['employment_duration'].map(attribute7_dic)
    df_num['installment_rate']=df_german['installment_rate']
    df_num['personal_status_and_sex']=df_german['personal_status_and_sex'].map(attribute9_dic)
    df_num['other_debtors']=df_german['other_debtors'].map(attribute10_dic)
    df_num['residence_duration']=df_german['residence_duration']
    df_num['property']=df_german['property'].map(attribute12_dic)
    df_num['age']=df_german['age']
    df_num['installment_plans']=df_german['installment_plans'].map(attribute14_dic)
    df_num['housing']=df_german['housing'].map(attribute15_dic)
    df_num['number_of_credits']=df_german['number_of_credits']
    df_num['job']=df_german['job'].map(attribute17_dic)
    df_num['people_liable']=df_german['people_liable']
    df_num['telephone']=df_german['telephone'].map(attribute19_dic)
    df_num['foreign_worker']=df_german['foreign_worker'].map(attribute20_dic)
    
    df_num['credit_risk']=df_german['credit_risk']
    df_num['predicted_label']=df_german['predicted_label'].apply(np.int64)
    df_num.head(10)
    return df_num

In [ ]:
def fair_metrics(dataset, ground_truth, sensitive):
    
    privileged_groups = [{sensitive: 1}]    
    unprivileged_groups = [{sensitive: 0}]

    
    binary = BinaryLabelDatasetMetric(dataset, 
                                       unprivileged_groups=unprivileged_groups, 
                                       privileged_groups=privileged_groups)
    disp_impact =  binary.disparate_impact().round(2) # Pr(Y = 1 | D = \text{unprivileged})/Pr(Y = 1 | D = \text{privileged})
    print(disp_impact)


    
    dataset_pred = ground_truth
    dataset_pred.labels = dataset.labels
    classif = ClassificationMetric(
                                  ground_truth,
                                  dataset_pred,
                                  unprivileged_groups=unprivileged_groups, 
                                  privileged_groups=privileged_groups    
                                  )

    fnrr = classif.false_negative_rate_difference()
    print(fnrr)


    return disp_impact, fnrr

In [ ]:
def calculate_freq_v_freq(df_num, mode, sensitive_list, threshold, business_list):
    #display(df_num.head())
    results = {}
    results_aif = {}
    fnrr_results_aif = {}
    results_aggr = []
    expl_list = []
    id_list = []
    dict_sens = {'foreign_worker': [2], 
                 'personal_status_and_sex': [4], 
                 'age_group': [4], 
                 'gender': [1],              
                 'marital_status': [0]
                } 
    skip_these_features = ['id', 'predicted_label', 'credit_risk', 'age']
    
    if t == None:
         bd = FreqVsFreqBiasDetector(distance="TVD", A1='high')
    else: 
        bd = FreqVsFreqBiasDetector(distance="TVD")
    hc = HazardFromBiasDetectionCalculator()
    
    #business_list = ['status', 'duration', 'credit_history'] #[c for c in df_num.columns if (c not in skip_these_features and c not in sensitive_list)]
    #print(business_list)

    # explainability
    expl = {}
    for variable in df_num.columns:
       # print(variable)
        if variable in skip_these_features:
            continue

        demo_parity = bd.compare_root_variable_groups(
            dataframe=df_num,
            target_variable='predicted_label',
            root_variable=variable,
            threshold=threshold
        )   
        expl[variable] = demo_parity['distance']

    
    expl = pd.DataFrame(expl.items(), columns=['Feature', 'Distance']).sort_values(by='Distance', ascending=False)

    if business_list == None:
            business_list = expl['Feature'].head(3).values
    expl_df = expl[expl['Feature'].isin(business_list)].reset_index(drop=True)
    print('Conditioning features: ' + str(business_list))
    
    for sensitive in sensitive_list:
        # demographic parity
        demo_parity = bd.compare_root_variable_groups(
            dataframe=df_num,
            target_variable='predicted_label',
            root_variable=sensitive,
            threshold=threshold
        )   
        
        #conditional demographic parity
        print('---------' + sensitive)
        cond_demo_parity = bd.compare_root_variable_conditioned_groups(
            dataframe=df_num,
            target_variable='predicted_label',
            root_variable=sensitive,
            conditioning_variables=business_list,
            threshold=threshold,
            min_obs_per_group=30
            )        
        print(demo_parity)
        #print(cond_demo_parity)
        
        # # intersectional fairness
        # intersectional = bd.compare_root_variable_conditioned_groups(
        # dataframe=df_num,
        # target_variable='predicted_label',
        # root_variable=sensitive,
        # conditioning_variables=[s for s in sensitive_list if s != sensitive],
        # )
        # print(intersectional)
        # results.append(intersectional)

        # hazard cioè rischio del singolo test
        hazard = hc.compute_hazard_from_freqvsfreq_or_freqvsref(
            demo_parity,
            cond_demo_parity,
            df_num.shape[0],
            business_list,
            weight_logic=mode
        )
        results[sensitive] = hazard
        results_aggr.append(hazard)
                  
        
        dataset_orig = StandardDataset(df_num,
                                       label_name='predicted_label',
                                       protected_attribute_names=[sensitive],
                                       privileged_classes=[lambda x: 1 if x == dict_sens[sensitive][0] else 0], 
                                       favorable_classes=[0]
                                        )       

        ground_truth = StandardDataset(df_num,
                               label_name='credit_risk',
                               protected_attribute_names=[sensitive],
                               privileged_classes=[lambda x: 1 if x == dict_sens[sensitive][0] else 0], 
                               favorable_classes=[0]
                                )              
        
        print(fair_metrics(dataset_orig, ground_truth, sensitive))
        a, b = fair_metrics(dataset_orig, ground_truth, sensitive)
        results_aif[sensitive] = a
        fnrr_results_aif[sensitive] = b
        
        if fair_metrics(dataset_orig, ground_truth, sensitive)[0]<1: #this means that the unprivileged class is disadvantaged
            display(df_num[~df_num[sensitive].isin(dict_sens[sensitive])].head())            
            exposure = df_num[~df_num[sensitive].isin(dict_sens[sensitive])]['id'].values
            id_list.append(exposure)
            print(exposure)
            
        elif fair_metrics(dataset_orig, ground_truth, sensitive)[0]>1:
            display(df_num[df_num[sensitive].isin(dict_sens[sensitive])].head())
            exposure = df_num[df_num[sensitive].isin(dict_sens[sensitive])]['id'].values
            id_list.append(exposure)
            print(exposure)
            
    print('intersection on ' +  str(len(id_list)) + ' sets:')
    print(len(list(reduce(lambda x, y: set(x).intersection(y), id_list))))
    
    print('vuln_non_pesata' + str(len(list(reduce(lambda x, y: set(x).intersection(y), id_list)))/df_num.shape[0]))
    vuln = len(list(reduce(lambda x, y: set(x).intersection(y), id_list)))/df_num.shape[0]*len(id_list)
    
    #print('Results for ' + mode + ' mode')
    # print(results)
    print(results_aggr)
    print('vuln ' + str(vuln))

    return results_aggr, results, results_aif, fnrr_results_aif, business_list, sensitive_list, vuln, expl_df



In [ ]:
# total_german = process_german_credit(total)
# total_german.to_csv(output_data_path + 'processed_german_credit_crif.csv', index=False, sep=',')  

In [ ]:
def plot_expl(list, df_num, i, label):

    for feat in list:
        
        # Create a figure and two subplots on the same row
        fig, axes = plt.subplots(1, 2, figsize=(8,4))  # 1 row, 2 columns, adjust figsize as needed
        
        # First plot (histogram)
        sns.histplot(
            data=df_num, 
            x=feat, 
            hue='predicted_label', 
            multiple="dodge", 
            palette={1: 'red', 0: 'blue'},
            alpha=0.5,
            ax=axes[0]  # Assign to the first subplot
        )
        axes[0].set_title('Histogram')
        
        # Second plot (bar plot)
        df_num.groupby([feat])['predicted_label'].mean().plot.bar(
            ax=axes[1],
            color='green'
            # Assign to the second subplot
    
        )
        axes[1].set_title('Mean risk score')
    
        fig.savefig(output_data_path + '/images/' + f'{label}_{feat}_model0' + str(i+1))
        # Adjust layout to prevent overlapping
        plt.tight_layout()
        plt.show()
    

In [ ]:
#prime 3 features per correlation (se la lista è None, verranno scelte le 3 feature con gli hazard più alti)
business_list_by_model = {0:['status', 'duration', 'credit_history'],
                          1:['status', 'duration', 'credit_history'],
                          2:['status', 'property', 'credit_history'],
                          3:['status', 'duration', 'credit_history'],
                          4:['credit_history', 'number_of_credits', 'employment_duration'],
                          5:None
                         }

# business_list_by_model = {0:None,
#                           1:None,
#                           2:None,
#                           3:None,
#                           4:None,
#                           5:None
#                          } 

for t in [0.15]:
    data = []
    list_group_fairness = []
    list_ind_fairness = []
    aif_table = []
    condit_feat = []
    sensitive_list = ['foreign_worker', 'personal_status_and_sex', 'age_group']
    threshold=t
    modes = ['group']#, 'individual']
    
    for i in range(6):
    
        print('************* MODEL 0' + str(i+1) )    
        
        dict = {}
        lista = {}
        condit_dict = {}
        rc = RiskCalculator()
        if i == 5:
            model = pd.read_csv(output_data_path + 'processed_model0' + str(i+1) + '.csv').sort_values(by='id')
            df_num = model.copy()
    
        else:
            model = pd.read_csv(input_data_path + 'model0' + str(i+1) + '.csv').sort_values(by='id')
            # print(model.shape)
            print(model['predicted_label'].value_counts(normalize=True))
            model = pd.merge(left=total, right=model, how='left', on='id')
            model = model.dropna(subset=['predicted_label'])
            df_num = process_german_credit(model)
            df_num = df_num.rename(columns={"savings/bonds": "savings_bonds"})

        #display(df_num.head())
        #df_num.to_csv(output_data_path + 'processed_model0' + str(i+1) + '.csv', index=False, sep=',')  
    
    
        # df_num.hist()
        # pyplot.rcParams['figure.figsize']=[16,16]
        # pyplot.show()
    
        # plt.figure(figsize=(10, 5))
        # df_num.groupby(['personal_status_and_sex'])['predicted_label'].mean().plot.bar()
        # plt.show()
        # df_num.groupby(['gender'])['predicted_label'].mean().plot.bar()
        # plt.show()
        # df_num.groupby(['age_group'])['predicted_label'].mean().plot.bar()
        # plt.show()
        # df_num.groupby(['foreign_worker'])['predicted_label'].mean().plot.bar()
        # plt.show()
    
        accuracy = (df_num['credit_risk'] == df_num['predicted_label']).mean()
        dict['accuracy'] = accuracy
        P = (df_num['predicted_label'] == 0).sum()  # Count where credit_risk is 1 (positive)
        N = (df_num['predicted_label'] == 1).sum()  # Count where credit_risk is 0 (negative)
        base_rate = P / (P + N)
        dict['acceptance_rate'] = base_rate
        print(business_list_by_model[i])
        for mode in modes:
            print('>>>>>>>>>>>>>>>>>>>>>>>>>' + mode + ' mode:')
            hazards_aggr, hazards, aif, fnrr_aif, expl, sensitive_list, vuln, expl_df = calculate_freq_v_freq(df_num, mode, sensitive_list, threshold, business_list = business_list_by_model[i])
            risk = rc.compute_risk([item[0] for item in hazards_aggr])
            print(risk)        
            dict[f'{mode}_risk'] = risk
            dict[f'{mode}_vulnerability'] = vuln
            if mode == 'group':
                list_group_fairness.append(hazards)
            else: 
                list_ind_fairness.append(hazards)
                
        condit_feat.append(expl_df)
        plot_expl(expl, df_num, i, 'expl')
        plot_expl(sensitive_list, df_num, i, 'sensitive')

        # revenue analysis    
        total_credit_amount=sum(df_num[(df_num['predicted_label']==0)]['credit_amount'])
        # bad_rate=len(df_num[(df_num['performance']==1) & (df_num['score_bin']==0)])/len(df_num[(df_num['score_bin']==0)])
        bad_rate=len(df_num[(df_num['predicted_label']==0) & (df_num['credit_risk']==1)])/len(df_num[(df_num['predicted_label']==0)])
        provision=total_credit_amount*bad_rate*0.2
        interest_rate = 1 #todo change this
        profit = round(total_credit_amount*interest_rate-provision, 0)
        dict['profit'] = profit
        data.append(dict)
        aif_table.append(aif)
        #aif_table.append(fnrr_aif)
    
    print('individual mode')
    ind_haz = pd.DataFrame(list_ind_fairness)
    for col in ind_haz.columns:
        ind_haz[f'{col}_relative_haz'] = ind_haz[col].apply(lambda x: x[0]/x[-1])
        ind_haz[f'{col}_absolute_haz'] = ind_haz[col].apply(lambda x: x[0])
    cols_rel = [col for col in ind_haz.columns if "relative" in col]
    cols_abs = [col for col in ind_haz.columns if "absolute" in col]
    ind_haz['overall_relative_haz'] =  ind_haz[cols_rel].mean(axis=1).apply(lambda x: '{:,.2%}'.format(x))
    ind_haz['overall_absolute_haz'] =  ind_haz[cols_abs].mean(axis=1)

    display(ind_haz)
    
    print('group mode')
    group_haz = pd.DataFrame(list_group_fairness)
    for col in group_haz.columns:
        group_haz[f'{col}_relative_haz'] = group_haz[col].apply(lambda x: x[0]/x[-1])
        group_haz[f'{col}_absolute_haz'] = group_haz[col].apply(lambda x: x[0])
    cols_rel = [col for col in group_haz.columns if "relative" in col]
    cols_abs = [col for col in group_haz.columns if "absolute" in col]
    group_haz['overall_relative_haz'] =  group_haz[cols_rel].mean(axis=1).apply(lambda x: '{:,.2%}'.format(x))

    group_haz['overall_absolute_haz'] =  group_haz[cols_abs].mean(axis=1)

    display(group_haz)
    
    print('comparison')
    results = pd.DataFrame(data)
    display(results)
    
    print('AIF360 demo_parity (p(unpriv)/p(priv))')
    aif_final = pd.DataFrame(aif_table)
    display(aif_final)
    
    print('Most conditioning factors')
    expl = pd.concat(condit_feat, axis=1)
    display(expl)

    with pd.ExcelWriter(output_data_path + f'threshold_{t}_results.xlsx') as writer:  
        aif_final.to_excel(writer, sheet_name='AIF360_comparison')
        results.to_excel(writer, sheet_name='results_overall')    
        group_haz.to_excel(writer, sheet_name='group_mode_hazards')
        ind_haz.to_excel(writer, sheet_name='individual_mode_hazards')
        expl.to_excel(writer, sheet_name='explainability')
    
